## import data

In [3]:
## import data
import numpy as np
import pandas as pd

data = pd.read_csv('pheno_allgenic.csv')
data_prepost = data[pd.notnull(data['SRS.prepost.cat'])]

x_prepost = data_prepost.iloc[:, list(range(3, 30))]
y_prepost = data_prepost.iloc[:, 1]   


## Support vector machine with rbf kernel

In [17]:
## try rbf SVM model for training. For feature selection, use sequential forward selection method to select most important feature set in each inner CV and calculate the total number of each features after 10-time of outer CV.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
    
y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

feature_list = []
for train, test in outer_cv.split(x_prepost, y_prepost_array):
    #rbf SVC
    clf_outer = SVC(class_weight='balanced', kernel='rbf', random_state=0)  
    ##stepwise forward selection
    sfs = SFS(clf_outer, k_features=(1, 27), forward=True, floating=True, verbose=0, scoring='roc_auc',  cv=inner_cv)   
    sfs = sfs.fit(x_prepost[train], y_prepost_array[train])
    feature_list.append(list(sfs.k_feature_idx_))


In [18]:
## summary the total select times for each feature after 10-time outer CV as importance measurement
feature_sum_dict = {}
feature_sum_sort_dict = {}
for i in feature_list:
    for j in i:
        if x_prepost.columns[j] not in feature_sum_dict:
            feature_sum_dict[x_prepost.columns[j]] = 1
        else:
            feature_sum_dict[x_prepost.columns[j]] += 1

## sort features in feature dict based on selected times
feature_sum_sort_dict = {k: v for k, v in sorted(feature_sum_dict.items(), key=lambda item: item[1], reverse=True)}
feature_sum_sort_dict

{'synapgene_ratio_comb': 8,
 'synapgene_asd0.01_common': 7,
 'treatment.kontakt': 7,
 'treatment.cbt': 5,
 'ndd_num': 5,
 'lof_cln': 5,
 'ados.tot': 4,
 'comorbid.anxiety': 4,
 'comorbid.other': 4,
 'cnv_size': 4,
 'BIN3_all_recode': 4,
 'age.pre': 3,
 'SRS.pre.p': 3,
 'cnvgene_num': 3,
 'X1.0adhd': 2,
 'damage_num': 2,
 'abas.f.total.pre': 2,
 'treatment.councel': 2,
 'wisc.fsiq': 1,
 'comorbid.depression': 1,
 'X0.5asd': 1,
 'ddcgas.pre': 1,
 'treatment.pharma': 1}

In [29]:
## choose the most selected features in SVM for prediction. Use gridsearchCV to optimize parameters
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc  

y_prepost_array = y_prepost.to_numpy()
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

fpr_train_dict = {}
tpr_train_dict = {}
fpr_test_dict = {}
tpr_test_dict = {}
auc_train_dict = {}
auc_test_dict = {}
accuracy_train_dict = {}
accuracy_test_dict = {}

c_dict = {}
gamma_dict = {}
feature_totalselect_dict = {}

#rbf SVC
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1, 10]}] 
svm = SVC(kernel='rbf', class_weight='balanced')
for i in range(2,8):   
    feature_list = []
    feature_select_dict = {}
    feature_select_dict={a:u for a, u in feature_sum_sort_dict.items() if u > i}
    for b in feature_select_dict:
        feature_list.append(list(x_prepost.columns).index(b)) 
    x_prepost_sel = x_prepost.iloc[:, feature_list]
    
    c_dict['feature select time > %i'%i]=[]
    gamma_dict['feature select time > %i'%i]=[]
        
    fpr_train_dict['feature select time > %i'%i]=[]  ## fpr: sensitivity
    tpr_train_dict['feature select time > %i'%i]=[]  ## 1-tpr: specificity
    fpr_test_dict['feature select time > %i'%i]=[]
    tpr_test_dict['feature select time > %i'%i]=[]
    auc_train_dict['feature select time > %i'%i]=[]
    auc_test_dict['feature select time > %i'%i]=[]
    accuracy_train_dict['feature select time > %i'%i]=[]
    accuracy_test_dict['feature select time > %i'%i]=[]
    feature_totalselect_dict['feature select time > %i'%i] = list(feature_select_dict.keys())
        
    for train, test in outer_cv.split(x_prepost_sel, y_prepost_array):
        clf = GridSearchCV(estimator=svm, param_grid=tuned_parameters, scoring='roc_auc', cv=inner_cv)
        clf.fit(x_prepost_sel[train], y_prepost_array[train])
        #rbf
        clf_outer = SVC(class_weight='balanced', C=clf.best_params_['C'], kernel='rbf', gamma=clf.best_params_['gamma'], random_state=0)  
        
        clf_outer.fit(x_prepost_sel[train], y_prepost_array[train])
        predicted, y_pred = clf_outer.predict(x_prepost_sel[train]), clf_outer.predict(x_prepost_sel[test])

        fpr_train, tpr_train, thresholds_train = roc_curve(y_prepost_array[train], predicted)
        fpr_test, tpr_test, thresholds_test = roc_curve(y_prepost_array[test], y_pred)
        auc_train, auc_test = auc(fpr_train, tpr_train), auc(fpr_test, tpr_test)
        accuracy_train, accuracy_test = accuracy_score(y_prepost_array[train], predicted), accuracy_score(y_prepost_array[test], y_pred)

        c_dict['feature select time > %i'%i].append(clf.best_params_['C'])
        gamma_dict['feature select time > %i'%i].append(clf.best_params_['gamma'])
        
        fpr_train_dict['feature select time > %i'%i].append(fpr_train[1])
        tpr_train_dict['feature select time > %i'%i].append(tpr_train[1])
        fpr_test_dict['feature select time > %i'%i].append(fpr_test[1])
        tpr_test_dict['feature select time > %i'%i].append(tpr_test[1])
        auc_train_dict['feature select time > %i'%i].append(auc_train)
        auc_test_dict['feature select time > %i'%i].append(auc_test)
        accuracy_train_dict['feature select time > %i'%i].append(accuracy_train)
        accuracy_test_dict['feature select time > %i'%i].append(accuracy_test)

In [10]:
## calculate average auc train and test value in outer CV
auc_avg_test_dict = {}
auc_avg_train_dict = {}
for key in auc_test_dict:
    auc_test, auc_train = 0.0, 0.0
    for time in auc_test_dict[key]:
        auc_test += time
    for time in auc_train_dict[key]:
        auc_train += time
    auc_avg_test, auc_avg_train = auc_test/10, auc_train/10
    auc_avg_test_dict[key]=auc_avg_test
    auc_avg_train_dict[key]=auc_avg_train
print(auc_avg_test_dict)


{'feature select time > 2': 0.5294444444444444, 'feature select time > 3': 0.6040277777777778, 'feature select time > 4': 0.5488888888888889, 'feature select time > 5': 0.584861111111111, 'feature select time > 6': 0.584861111111111, 'feature select time > 7': 0.5298611111111111}
